In [39]:
import io
import os
import sys
import arcpy
import laspy
from zipfile import ZipFile
import zipfile
import pandas as pd
import requests
import ftplib
from bs4 import BeautifulSoup as bs
import datetime
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureCollection, FeatureSet

In [2]:
# Directory where the .bil files will be saved
save_folder = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\bil_files"
os.makedirs(save_folder, exist_ok=True)  # Create the folder if it doesn't exist

# URL to download PRISM data
url = "https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip"

# Fetch the data
response = requests.get(url)

# Specify the output file name
output_zip_file = os.path.join(save_folder, "prism_data.zip")

# Save the file
with open(output_zip_file, 'wb') as file:
    file.write(response.content)

print(f"Data saved successfully to {output_zip_file}")

# Unzip the file using Python's zipfile module
with zipfile.ZipFile(output_zip_file, 'r') as zip_ref:
    zip_ref.extractall(save_folder)

print(f"Data extracted successfully to {save_folder}")

Data saved successfully to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\bil_files\prism_data.zip
Data extracted successfully to C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\bil_files


In [44]:
# Define the geodatabase path, mosaic dataset name, and spatial reference (EPSG 4269 - NAD83)
arcpy.management.CreateMosaicDataset(
    in_workspace=r"C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2\mosaic_lab2.gdb",
    in_mosaicdataset_name="Lab2_MosaicDS",
    coordinate_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

print(f"Mosaic dataset '{mosaic_name}' created successfully in {gdb_path}")

Mosaic dataset 'Lab2_Mosaic' created successfully in C:\Users\ethan\Documents\ArcGIS\Projects\GIS5571_Lab2\mosaic_lab2.gdb


In [47]:
#Add the .bil datasets to the Created Mosaic Dataset in the geodatabase
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="Lab2_MosaicDS",
    raster_type="Raster Dataset",
    input_path=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\bil_files",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=4269,
    filter="*.bil",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="ESTIMATE_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\ethan\AppData\Local\ESRI\rasterproxies\Lab2_MosaicDS"
)

<Result 'Lab2_MosaicDS'>

In [48]:
arcpy.management.CalculateField(
    in_table=r"Lab2_MosaicDS\Footprint",
    field="Variable",
    expression='"30yr Normal"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Lab2_MosaicDS\\Footprint'>

In [49]:
arcpy.management.CalculateField(
    in_table=r"Lab2_MosaicDS\Footprint",
    field="Timestamp",
    expression="datetime.datetime(2023, int(!Name![-6:-4]), 1) if !Name![-6:-4].isdigit() else None",
    expression_type="PYTHON3",
    code_block="import datetime",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Lab2_MosaicDS\\Footprint'>

In [50]:
arcpy.management.BuildMultidimensionalInfo(
    in_mosaic_dataset="Lab2_MosaicDS",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'Lab2_MosaicDS'>

In [51]:
#Make multidimensional raster layer
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="Lab2_MosaicDS",
    out_multidimensional_raster_layer="Lab2_MosaicDS_MultidimLayer",
    variables="'30yr Normal'",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

<Result 'Lab2_MosaicDS_MultidimLayer'>

In [52]:
# Create Space-Time Cube from Multidimensional Rastser
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="Lab2_MosaicDS_MultidimLayer",
    output_cube=r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\STCube\STCube.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Users\\ethan\\Desktop\\ARLT_MGIS\\GIS5571\\Lab2\\STCube\\STCube.nc'>

In [55]:
# Set up the output path for the animation
output_animation_path = r"C:\Users\ethan\Desktop\ARLT_MGIS\GIS5571\Lab2\STCube\PPT_30Norms.avi"  # Specify your output format and path

# Define the view to export (usually the active view in the project)
active_view = aprx.activeView

# Export animation settings
arcpy.mp.ExportAnimation(
    in_view=active_view,
    out_file=output_animation_path,
    export_type="MP4",  
    resolution="1280x720"  # Set the desired resolution
)

print(f"Animation exported successfully to: {output_animation_path}")

AttributeError: module 'arcpy.mp' has no attribute 'ExportAnimation'